## Case Study

**Scenario**: Instacart has provided user purchase data and wants to create user behavior profilesv to target their marketing.

**Example**: the power user, or organic andy - and be able to describe their behavior and how it differs from other groups.

**Deliverable**: 
- A paragraph with a nice looking table describing the different user groups in the dataset.
- The table will be produced through the clustering algorithms you learned today.
- You will cluster on `user_summaries`

**Data**: Download the zip file from [this link](https://drive.google.com/file/d/1_nl_wVnYYNlYe1cWlVU3QAWAWyPxS4S8/view?usp=sharing), unzip the folder, and save all the contents in a file named "data"

**Tasks**:
1. Create the following aggregate variables to add to your user summaries:

- average basket size
- percent of orders that contain organic items
- average percent of basket that is organic

2. create at least 3 other new aggregate variables (unique from other people in the group) from the `full_order_data` to add to `user_summaries`

3. Run both hierarchical and kmeans clustering to find the optimum # of groups

4. Write a paragraph to describe those cluster attributes

5. Make sure the table looks "client ready"

In [9]:
!ls ./data

aisles.csv.zip                orders.csv.zip
departments.csv.zip           products.csv.zip
order_products__prior.csv.zip sample_submission.csv.zip
order_products__train.csv.zip


In [1]:
# Import necessary data manipulation packages

import pandas as pd
pd.options.display.max_columns = 999

import numpy as np
import zipfile

import matplotlib.pyplot as plt
%matplotlib inline

### Orders

In [2]:
zf = zipfile.ZipFile('./data/orders.csv.zip')
orders = pd.read_csv(zf.open(zf.namelist()[0]))

In [3]:
# Create aggregates of order count and days between orders

max_orders = orders.groupby('user_id')['order_number'].max()
avg_days = orders.groupby('user_id')['days_since_prior_order'].mean()
max_days = orders.groupby('user_id')['days_since_prior_order'].max()
min_days = orders.groupby('user_id')['days_since_prior_order'].min()

In [4]:
# create new user_summary dataset
user_summaries = pd.DataFrame([avg_days,max_days, min_days, max_orders]).T

names = ['av_days_btw_orders', 'max_days_btw_orders', 'min_days_btw_orders', 'total_orders']
user_summaries.columns = names

### Products

In [5]:
zf = zipfile.ZipFile('./data/products.csv.zip')
products = pd.read_csv(zf.open(zf.namelist()[0]))

In [6]:
# create organic tag for each product
products['organic'] = np.where(products.product_name.str.contains("Organic"),
                             1, 0)
# remove Organic from product names
products.product_name=products.product_name.str.replace("Organic","")

### Department

In [7]:
zf = zipfile.ZipFile('./data/departments.csv.zip')
department = pd.read_csv(zf.open(zf.namelist()[0]))

### Order Products


In [8]:
zf = zipfile.ZipFile('./data/order_products__prior.csv.zip')
prior_orders = pd.read_csv(zf.open(zf.namelist()[0]))


In [9]:
zf = zipfile.ZipFile('./data/order_products__train.csv.zip')
order_prods = pd.read_csv(zf.open(zf.namelist()[0]))

In [10]:
order_prods = prior_orders.append(order_prods)
order_prods.sort_values('order_id',inplace=True)

## Create unified table

In [11]:
order_user_key = orders[["order_id", "user_id"]]

# merge user_id on to order table
full_table = order_prods.merge(order_user_key, on = 'order_id', how ='left') 

# merge product info on to table
full_table_copy = full_table.merge(products, on="product_id", how='left')

# merge department info on to table
full_order_data = full_table_copy.merge(department, on="department_id", how='left')

## What is the average basket size by user?

### Get Basket Size 

In [12]:
# make a groupby with order id on full_order_data to get count of products in each order
order_size = full_order_data.groupby(["order_id"])["order_id"].count()

# make it a nicer dataframe
order_size_df = order_size.to_frame()
order_size_df.columns = ['basket_size']
order_size_df.reset_index(inplace=True)

### Get average basket count

In [13]:
# merge on order_id
get_avg_basket = order_user_key.merge(order_size_df, on="order_id", how = 'inner')

# get groupby on user_id
avg_basket_user = get_avg_basket.groupby('user_id')['basket_size'].mean()

# Make it a nicer data frame
avg_basket_user_df = avg_basket_user.to_frame()
avg_basket_user_df.reset_index(inplace = True)
avg_basket_user_df.columns = ["user_id", "average_basket_size"]

## Get count of organic items by basket

There was more than one way to do this, this is only the path *I* chose

### Get count of organic items by basket

In [14]:
full_order_data.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,product_name,aisle_id,department_id,organic,department
0,1,49302,1,1,112108,Bulgarian Yogurt,120,16,0,dairy eggs
1,1,10246,3,0,112108,Celery Hearts,83,4,1,produce
2,1,49683,4,0,112108,Cucumber Kirby,83,4,0,produce
3,1,43633,5,1,112108,Lightly Smoked Sardines in Olive Oil,95,15,0,canned goods
4,1,13176,6,0,112108,Bag of Bananas,24,4,1,produce


In [15]:
organic_orders = full_order_data.groupby(["order_id"])["organic"].sum()

# make it a nicer dataframe
organic_orders_df = organic_orders.to_frame()
organic_orders_df.columns = ['organic_items']
organic_orders_df.reset_index(inplace=True)

In [16]:
get_organic_product_counts = order_user_key.merge(organic_orders_df, on="order_id", how = 'left')

In [17]:
# Prepare for aggregating

# merge count of organic items onto count of items in basket at all
organic_product_dems = order_size_df.merge(get_organic_product_counts, on = "order_id", how ='left')
# sort values of DF by user_id and order_id for readability
organic_product_dems.sort_values(by=['user_id',"order_id"], inplace=True)

# create new variable "percent" of basket that is organic
organic_product_dems["percent"] = organic_product_dems.organic_items/organic_product_dems.basket_size

In [18]:
# get groupbys by user of mean, min, and max percent organic in their basket history
# then change the name each series appropriately so it is the column name when joined together in a dataframe

mean_org= organic_product_dems.groupby('user_id')['percent'].mean()
mean_org.name = "org_mean_percent"

min_org = organic_product_dems.groupby('user_id')['percent'].min()
min_org.name = "org_min_percent"

max_org = organic_product_dems.groupby('user_id')['percent'].max()
max_org.name = "org_max_percent"

In [19]:
org_desc_dict = {mean_org.name:mean_org, min_org.name:min_org, max_org.name:max_org}
org_desc_df = pd.DataFrame(org_desc_dict)
org_desc_df.reset_index(inplace=True)

### Get number of orders WITH any organic items in the basket

In [20]:
get_organic_order_counts_series = get_organic_product_counts.loc[get_organic_product_counts.organic_items > 0].groupby('user_id')['order_id'].count()

In [21]:
get_organic_order_counts_series

user_id
1         11
2         15
3         11
4          1
5          5
          ..
206205     4
206206    40
206207    16
206208    49
206209     9
Name: order_id, Length: 190566, dtype: int64

In [22]:
get_organic_order_counts_df = get_organic_order_counts_series.to_frame()

In [23]:
get_organic_order_counts_df.reset_index(inplace = True)

In [24]:
get_organic_order_counts_df.columns = ['user_id', "organic_basket_count"]

In [25]:
get_organic_order_counts_df.shape

(190566, 2)

## Enhance User Summaries table

In [26]:
user_summaries.reset_index(inplace=True)

In [27]:
user_summaries.shape

(206209, 5)

In [28]:
user_summaries=user_summaries.merge(avg_basket_user_df, on = "user_id", how ="left")

In [90]:
user_summaries

,user_id,av_days_btw_orders,max_days_btw_orders,min_days_btw_orders,total_orders,average_basket_size,organic_basket_count
0,1,19.000000,30.0,0.0,11.0,11.727273,11.0
1,2,16.285714,30.0,3.0,15.0,28.066667,15.0
2,3,12.000000,21.0,7.0,13.0,14.666667,11.0
3,4,17.000000,30.0,0.0,6.0,7.200000,1.0
4,5,11.500000,19.0,6.0,5.0,16.600000,5.0
...,...,...,...,...,...,...,...
206204,206205,16.666667,30.0,10.0,4.0,20.750000,4.0
206205,206206,3.716418,15.0,0.0,68.0,8.507463,40.0
206206,206207,14.312500,30.0,1.0,17.0,27.875000,16.0
206207,206208,7.367347,20.0,0.0,50.0,27.632653,49.0


In [29]:
user_summaries=user_summaries.merge(get_organic_order_counts_df, on = "user_id", how ="left")

In [30]:
user_summaries['perc_organic_orders'] = user_summaries.organic_basket_count/user_summaries.total_orders

In [32]:
user_summaries

,user_id,av_days_btw_orders,max_days_btw_orders,min_days_btw_orders,total_orders,average_basket_size,organic_basket_count,perc_organic_orders
0,1,19.000000,30.0,0.0,11.0,6.363636,11.0,1.000000
1,2,16.285714,30.0,3.0,15.0,15.066667,15.0,1.000000
2,3,12.000000,21.0,7.0,13.0,7.333333,11.0,0.846154
3,4,17.000000,30.0,0.0,6.0,3.600000,1.0,0.166667
4,5,11.500000,19.0,6.0,5.0,9.200000,5.0,1.000000
...,...,...,...,...,...,...,...,...
206204,206205,16.666667,30.0,10.0,4.0,12.750000,4.0,1.000000
206205,206206,3.716418,15.0,0.0,68.0,4.253731,40.0,0.588235
206206,206207,14.312500,30.0,1.0,17.0,13.937500,16.0,0.941176
206207,206208,7.367347,20.0,0.0,50.0,13.816327,49.0,0.980000


In [31]:
user_summaries = user_summaries.merge(org_desc_df, on = "user_id", how ="left")

In [32]:
user_summaries

,user_id,av_days_btw_orders,max_days_btw_orders,min_days_btw_orders,total_orders,average_basket_size,organic_basket_count,perc_organic_orders,org_mean_percent,org_min_percent,org_max_percent
0,1,19.000000,30.0,0.0,11.0,6.363636,11.0,1.000000,0.250298,0.166667,0.375000
1,2,16.285714,30.0,3.0,15.0,15.066667,15.0,1.000000,0.255468,0.076923,0.400000
2,3,12.000000,21.0,7.0,13.0,7.333333,11.0,0.846154,0.339499,0.000000,0.600000
3,4,17.000000,30.0,0.0,6.0,3.600000,1.0,0.166667,0.057143,0.000000,0.285714
4,5,11.500000,19.0,6.0,5.0,9.200000,5.0,1.000000,0.485051,0.333333,0.636364
...,...,...,...,...,...,...,...,...,...,...,...
206204,206205,16.666667,30.0,10.0,4.0,12.750000,4.0,1.000000,0.430106,0.352941,0.571429
206205,206206,3.716418,15.0,0.0,68.0,4.253731,40.0,0.588235,0.228968,0.000000,0.916667
206206,206207,14.312500,30.0,1.0,17.0,13.937500,16.0,0.941176,0.528823,0.333333,0.750000
206207,206208,7.367347,20.0,0.0,50.0,13.816327,49.0,0.980000,0.430539,0.090909,0.750000


In [33]:
def user_trends(pattern):

    dem_orders = full_order_data.groupby(["order_id"])[pattern].sum()

    # make it a nicer dataframe
    dem_orders_df = dem_orders.to_frame()
    dem_orders_df.columns = ['{}_items'.format(pattern)]
    dem_orders_df.reset_index(inplace=True)

    get_dem_product_counts = order_user_key.merge(dem_orders_df, on="order_id", how = 'left')

    # Prepare for aggregating

    # merge count of organic items onto count of items in basket at all
    dem_product_dem = order_size_df.merge(get_dem_product_counts, on = "order_id", how ='left')
    # sort values of DF by user_id and order_id for readability
    dem_product_dem.sort_values(by=['user_id',"order_id"], inplace=True)
    
    print(dem_product_dem.columns)

    # create new variable "percent" of basket that is organic
    dem_product_dem["percent"] = dem_product_dem.iloc[:,-1:]/organic_product_dems.basket_size

    mean_dem= dem_product_dem.groupby('user_id')['percent'].mean()
    mean_dem.name = "mean_percent"

    min_dem = dem_product_dem.groupby('user_id')['percent'].min()
    min_dem.name = "min_percent"

    max_dem = dem_product_dem.groupby('user_id')['percent'].max()
    max_dem.name = "max_percent"

    dem_desc_dict = {mean_org.name:mean_org, min_org.name:min_org, max_org.name:max_org}
    dem_desc_df = pd.DataFrame(org_desc_dict)
    dem_desc_df.reset_index(inplace=True)
    
    return dem_desc_df

In [ ]:
user_trends('reordered')

Index(['order_id', 'basket_size', 'user_id', 'reordered_items'], dtype='object')
